In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/ARP_10_01_2022.pdf"
#pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 3

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan

The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track :  ARAP <AHOE PARK
date :  October 1, 2022
race_num :  4
criteria :  FOR THREE YEAR OLDS AND UPWARD.
claiming_price :  $2,500.
track_length :  Claiming Price $2,500 Four And One Half Furlongs On The Dirt
purse :  $9,000
weather :  Cloudy
track_type :  Good
off_time :  2 16
start :  Good for all
track : Arapahoe Park
date : October 1, 2022
race_num : 4
criteria : FOR THREE YEAR OLDS AND UPWARD
claiming_price : $2,500
track_length :  Four And One Half Furlongs On The Dirt
purse : $9,000
weather : Cloudy
track_type : Good
off_time : 2:16
start : Good for all
race_type : claiming
Looking for tables...
Found table top bound at 125.12199999999996
Found table bottom bound at 216.42199999999988
Looking for tables...
Found table top bound at 430.772
Found table bottom bound at 531.842
     Last Raced  Unnamed: 0  Pgm               Horse Name (Jockey)    Wgt  \
0           2 4         NaN  NaN                               NaN    NaN   
1    6Sep22 ARP         NaN  1.0    Getaloadofthis (H

In [4]:
result_tables[1]

,Pgm,Horse Name,Start,1/4,Str,Fin,1/4_length_behind,Str_length_behind,Fin_length_behind
0,1.0,Getaloadofthis,1.0,1,1,1,1/2,2,7
1,5.0,Its a Deal,3.0,2,3,2,1/2,2,7
2,2.0,Cherubim,5.0,6,6,3,5 1/2,5 3/4,8 1/2
3,8.0,Dryspell,4.0,4,4,4,2,2 1/4,9 1/4
4,6.0,All Shacked Up,2.0,3,2,5,2,2,9 1/4
5,3.0,Jack With Seven,8.0,5,5,6,3 1/2,3 3/4,11 1/4
6,7.0,El Diablo Grande,6.0,7,7,7,7,6 1/4,11 1/2
7,4.0,Zcat's Chatain,7.0,8,8,8,11,10 1/4,16 3/4


In [ ]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0].astype(object)
    bottom_table = result_tables[1].astype(object)
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 6}, {'page_num': 1, 'horse_count': 7}, {'page_num': 2, 'horse_count': 6}, {'page_num': 3, 'horse_count': 8}, {'page_num': 4, 'horse_count': 6}, {'page_num': 5, 'horse_count': 5}, {'page_num': 6, 'horse_count': 6}, {'page_num': 7, 'horse_count': 6}, {'page_num': 8, 'horse_count': 9}]
track :  ARAPAHOE PARK
date :  October 1, 2022
race_num :  1
criteria :  FOR THREE YEAR OLDS AND UPWARD NON WINNERS OF THREE RACES LIFETIME.
track_length :  Older, 124 lbs. (NW3 L) Seven Furlongs On The Dirt
purse :  $9,800
weather :  Cloudy
track_type :  Muddy
off_time :  1 01
start :  Good for all
{'track': 'Arapahoe Park', 'date': 'October 1, 2022', 'race_num': '1', 'criteria': 'FOR THREE YEAR OLDS AND UPWARD NON WINNERS OF THREE RACES LIFETIME', 'track_length': ' Seven Furlongs On The Dirt', 'purse': '$9,800', 'weather': 'Cloudy', 'track_type': 'Muddy', 'off_time': '1:01', 'start': 'Good for all', 'race_type': 'allowance'}
Looking for tables...
Found table top bound at 12

The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 443.52400000000006
     Last Raced  Unnamed: 0  Pgm                Horse Name (Jockey)   Wgt M/E  \
0           1 3         NaN  NaN                                NaN       NaN   
1   15Aug22 ARP         NaN  5.0         RB HASHTAG (Herrera, Hugo)    124 BL   
2           1 2         NaN  NaN                                NaN       NaN   
3   11Sep22 ARP         NaN  2.0   WMA SMOKE SIGNAL (Martin, David)  120 BL b   
4           4 5         NaN  NaN                                NaN       NaN   
5   25Sep22 ARP         NaN  4.0   UPTOWN RINGOFFIRE (Haar, Nathan)  124 BL b   
6           1 1         NaN  NaN                                NaN       NaN   
7   12Sep22 ARP         NaN  3.0            ROXY CS (Ramos, Adrian)  117 BL f   
8           1 1         NaN  NaN                                NaN       NaN   
9    8Aug22 ARP         NaN  6.0     RS JEWEL CROWN (Wales, Travis)  117 BL f   
10          1 7         NaN  NaN                              

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

Found table bottom bound at 495.799
     Last Raced  Unnamed: 0  Pgm                  Horse Name (Jockey)  \
0           9 5         NaN  NaN                                  NaN   
1   19Sep22 ARP         NaN  6.0           Three Ball (Wales, Travis)   
2           4 1         NaN  NaN                                  NaN   
3   20Sep22 ARP         NaN  4.0             Preacher (Ramos, Adrian)   
4           6 2         NaN  NaN                                  NaN   
5   12Sep22 ARP         NaN  2.0        Cougar Country (Haar, Nathan)   
6           4 2         NaN  NaN                                  NaN   
7   23Aug22 ARP         NaN  3.0        Gee Whiz Who (Cardoso, David)   
8           6 1         NaN  NaN                                  NaN   
9   12Sep22 ARP         NaN  1.0         Most Amusing (Herrera, Hugo)   
10          8 6         NaN  NaN                                  NaN   
11   6Sep22 ARP         NaN  5.0  West Coast Broker (Gonzalez, Pedro)   

     Wgt M/E  

Looking for tables...
Found table top bound at 117.24599999999997
Found table bottom bound at 188.54599999999994
Looking for tables...


C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_10_01_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table top bound at 394.18100000000004
Found table bottom bound at 468.815
    Last Raced  Unnamed: 0  Pgm                Horse Name (Jockey)    Wgt M/E  \
0          7 2         NaN  NaN                                NaN        NaN   
1  15Sep22 CBY         NaN  5.0    Tiz Wonderfully (Bethke, Scott)  124 BL bf   
2          1 3         NaN  NaN                                NaN        NaN   
3  19Sep22 ARP         NaN  4.0   Remarkable Charm (Wales, Travis)     124 BL   
4          5 2         NaN  NaN                                NaN        NaN   
5  25Sep22 ARP         NaN  3.0    Porsumsugaronme (Ramos, Adrian)     124 BL   
6          8 2         NaN  NaN                                NaN        NaN   
7  20Sep22 ARP         NaN  2.0  T. P. Right Lane (Cardoso, David)     124 BL   
8          1 2         NaN  NaN                                NaN        NaN   
9  19Sep22 ARP         NaN  7.0      Ultra Cordial (Herrera, Hugo)      124 b   

    PP  Start    1/4  1/2  S

Looking for tables...
Found table top bound at 116.53800000000001
Found table bottom bound at 197.83799999999997
Looking for tables...
Found table top bound at 402.765


In [8]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,Str 1,Str,Fin,Odds,...,1/2_length_behind_RLP,3/4_length_behind_RLP,3/16,3/8,3/16_length_behind,3/8_length_behind,3/16_RLP,3/8_RLP,3/16_length_behind_RLP,3/8_length_behind_RLP
0,29Aug22 ARP,3.0,Ramiro,Garcia,3.0,3.0,2,2,1,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28Aug22 ARP,2.0,Francisco,Giles,2.0,2.0,1,1,2,0.60*,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14Aug22 ARP,4.0,Juan,Rosales Gallegos,4.0,1.0,3,3,3,8.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29Aug22 ARP,1.0,Jhaciel,Soto,1.0,4.0,4,4,4,2.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14Aug22 ARP,5.0,Rey,Hernandez,5.0,5.0,5,5,5,12.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,11Sep22 ARP,4.0,Adrian,Ramos,4.0,6.0,NaN,5,2,7.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,6Sep22 ARP,6.0,Hugo,Herrera,5.0,1.0,NaN,1,3,4.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,6Sep22 ARP,2.0,Pedro,Gonzalez,2.0,5.0,NaN,6,4,33.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,11Sep22 ARP,3.0,Travis,Wales,3.0,2.0,NaN,4,5,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pdf_df.to_csv("ARAP.csv")